In [ ]:
from jetbot import Robot
import traitlets
import ipywidgets.widgets as widgets
import numpy as np

robot = Robot()

In [ ]:
image = widgets.Image(format='jpeg', width=300, height=300)

display(image)

In [ ]:
from jetbot import Camera

camera = Camera.instance()

In [ ]:
from jetbot import bgr8_to_jpeg

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

In [ ]:
from jetbot import Heartbeat


def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)

# attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

In [ ]:
import cv2
def preprocess(frame):
    frame = cv2.medianBlur(frame, 3)
    frame = cv2.addWeighted(frame, 1, np.zeros(frame.shape, frame.dtype), 0, 2)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return frame

In [ ]:
def get_two_line_centers(lines, img_width):
    """
    Collect all line midpoints, divide by vertical center (x-axis),
    and return mean point for left and right groups.
    """
    if lines is None or len(lines) == 0:
        return None, None
    
    # Collect all midpoints
    midpoints = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        mx = (x1 + x2) / 2
        my = (y1 + y2) / 2
        midpoints.append((mx, my))
    
    # Divide by vertical center (x-coordinate)
    center_x = img_width / 2
    left_points = [p for p in midpoints if p[0] < center_x]
    right_points = [p for p in midpoints if p[0] >= center_x]
    
    # Calculate mean for each group
    left_center = None
    right_center = None
    
    if left_points:
        left_x = np.mean([p[0] for p in left_points])
        left_y = np.mean([p[1] for p in left_points])
        left_center = (int(left_x), int(left_y))
    
    if right_points:
        right_x = np.mean([p[0] for p in right_points])
        right_y = np.mean([p[1] for p in right_points])
        right_center = (int(right_x), int(right_y))
    
    return left_center, right_center


In [ ]:
def calculate_steering(left_center, right_center, img_width, k=1.0):
    if left_center is not None and right_center is not None:
        lane_center_x = (left_center[0] + right_center[0]) / 2
        lane_center_y = (left_center[1] + right_center[1]) / 2
        lane_center = (int(lane_center_x), int(lane_center_y))
    
    # If only one line detected, estimate lane center
    elif left_center is not None:
        # Assume lane width and estimate right side
        estimated_lane_width = img_width * 0.4  # Adjust based on your lane
        lane_center_x = left_center[0] + estimated_lane_width / 2
        lane_center_y = left_center[1]
        lane_center = (int(lane_center_x), int(lane_center_y))
    
    elif right_center is not None:
        # Assume lane width and estimate left side
        estimated_lane_width = img_width * 0.4
        lane_center_x = right_center[0] - estimated_lane_width / 2
        lane_center_y = right_center[1]
        lane_center = (int(lane_center_x), int(lane_center_y))
    
    else:
        # No lines detected
        return 0.0, None
    
    # Calculate steering error
    image_center_x = img_width / 2
    error_x = lane_center[0] - image_center_x
    
    # Normalize to [-1, 1]
    max_offset = img_width / 2
    normalized_error = error_x / max_offset
    
    # Apply gain and clamp
    steering = k * normalized_error
    steering = max(-1.0, min(1.0, steering))
    
    return steering, lane_center

In [ ]:
class Kinematics:
    """Handle robot kinematics calculations"""
    
    def __init__(self, wheel_radius, wheel_distance, pulses_per_turn):
        self.wheel_radius = wheel_radius
        self.wheel_distance = wheel_distance
        self.pulses_per_turn = pulses_per_turn
    
    def wheel_speed_commands(self, u_desired, w_desired):
        wr_desired = float((2*u_desired + self.wheel_distance*w_desired) / (2*self.wheel_radius))
        wl_desired = float((2*u_desired - self.wheel_distance*w_desired) / (2*self.wheel_radius))
        return wl_desired, wr_desired
    
    def get_angular_and_linear_speed(self, wl, wr):

        u = self.wheel_radius / 2.0 * (wr + wl)
        w = self.wheel_radius / self.wheel_distance * (wr - wl)
        return u, w
    
    def get_wheel_speed(self, encoders, old_encoders, delta_t):
        ang_diff_l = 2*np.pi*(encoders[0] - old_encoders[0]) / self.pulses_per_turn
        ang_diff_r = 2*np.pi*(encoders[1] - old_encoders[1]) / self.pulses_per_turn
        
        wl = ang_diff_l / delta_t
        wr = ang_diff_r / delta_t
        
        return wl, wr
    
    def get_robot_pose(self, u, w, x_old, y_old, phi_old, delta_t):
        delta_phi = w * delta_t
        phi = phi_old + delta_phi
        
        # Normalize angle to [-pi, pi]
        if phi >= np.pi:
            phi = phi - 2*np.pi
        elif phi < -np.pi:
            phi = phi + 2*np.pi
        
        delta_x = u * np.cos(phi) * delta_t
        delta_y = u * np.sin(phi) * delta_t
        x = x_old + delta_x
        y = y_old + delta_y
        
        return x, y, phi
    
    def update_pose(self, encoder_values):
        wl, wr = self.kinematics.get_wheel_speed(
            encoder_values, 
            self.old_encoder_values, 
            self.delta_t
        )
        
        # Get robot speeds
        u, w = self.kinematics.get_angular_and_linear_speed(wl, wr)
        
        # Update pose
        self.x, self.y, self.phi = self.kinematics.get_robot_pose(
            u, w, self.x, self.y, self.phi, self.delta_t
        )
        
        # Update old encoder values
        self.old_encoder_values = encoder_values



In [ ]:
WHEEL_RADIUS = 0.325
WHEEL_DISTANCE = 0.15
PULSES_PER_TURN = 330

kinematics = Kinematics(WHEEL_RADIUS, WHEEL_DISTANCE, PULSES_PER_TURN)

In [ ]:
left_speed, right_speed = kinematics.wheel_speed_commands(0, 0)
robot.set_motors(left_speed, right_speed)

In [ ]:
prev_steering = 0.0
alpha = 0.2
while True:
    image_aux = preprocess(camera.value)
    h, w = camera.value.shape[:2]
    sliced_image = camera.value[int(h*0.5):h, :, :]

    CANNY_THRESHOLD = 70
    edges = cv2.Canny(image_aux, CANNY_THRESHOLD, CANNY_THRESHOLD * 1.1)

    h, w = edges.shape[:2]
    edges = edges[int(h*0.5):h, :]

    lines = cv2.HoughLinesP(
        edges,
        1,
        np.pi/180,
        50,
        minLineLength=10,
        maxLineGap=50
    )
    steering = 0.0
    steering_limited = 0.0
        
    if lines is not None:
        # Get the two mean centers
        left_center, right_center = get_two_line_centers(lines, sliced_image.shape[1])

        # Calculate steering angle
        steering, lane_center = calculate_steering(
            left_center, 
            right_center, 
            sliced_image.shape[1],
            k=2  # Adjust gain for your robot's responsiveness
        )
        prev_steering = (1 - alpha) * prev_steering + alpha * steering
        smoothed = prev_steering
        steering_limited = max(-0.5, min(0.5, steering))
    left_speed, right_speed = kinematics.wheel_speed_commands(0.1, steering_limited)
    robot.set_motors(left_speed, right_speed)

In [ ]:
robot.stop()
camera.stop()